<a href="https://colab.research.google.com/github/Heffx/CNN/blob/master/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
# Conferindo se o CUDA esta disponivel

verificaGPU = torch.cuda.is_available()

if not verificaGPU:
  print('CUDA INDISPONIVEL, TREINANDO COM CPU')
else:
  print('CUDA DISPONIVEL, TREINANDO COM GPU')

CUDA DISPONIVEL, TREINANDO COM GPU


In [3]:
!pip3 install torchvision

In [0]:
# Fazendo download e descompactando o dataset
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
import time
import json
import copy
import os
import flower_data
from torchvision import datasets
import torchvision
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import glob
import numpy as np
from skimage import data, io, filters


#Carregando, Normalizando e Criando vetor de Imagens

In [6]:
#Carregando treino com data loader
imagenet_train = torchvision.datasets.ImageFolder('flower_data/train/')

imagenet_valid = torchvision.datasets.ImageFolder('flower_data/valid/')
                                                 
                                                 
print(imagenet_train)
print(imagenet_valid)
data_loader_train = torch.utils.data.DataLoader(imagenet_train,
                                          batch_size=10,
                                          shuffle=True)

data_loader_valid = torch.utils.data.DataLoader(imagenet_valid,
                                          batch_size=10,
                                          shuffle=True)

print(data_loader_train)
print(data_loader_valid)



Dataset ImageFolder
    Number of datapoints: 6552
    Root Location: flower_data/train/
    Transforms (if any): None
    Target Transforms (if any): None
Dataset ImageFolder
    Number of datapoints: 818
    Root Location: flower_data/valid/
    Transforms (if any): None
    Target Transforms (if any): None


In [7]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print(data_transforms)

{'train': Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
), 'valid': Compose(
    Resize(size=256, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)}


In [0]:
def read_images(path):
  classes = glob.glob(path + '*')
  im_files = []
  size_classes = []
  for i in classes:
    # Agrupa as imagens por classe
    name_images_per_class = glob.glob(i+ '/*')
    im_files = im_files+name_images_per_class
    # Agrupa a quantidade de cada classe no vetor size_class
    size_classes.append(len(name_images_per_class))
  labels = np.zeros((len(im_files), len(classes)))
  ant = 0
  
  for id_i, i in enumerate(size_classes):
    labels[ant:ant+i, id_i] = 1
    ant = i
  collection = io.imread_collection(im_files)
  data = []
  
  for id_i, i in enumerate(collection):
    data.append(i)
  return np.asarray(data) , np.asarray(labels)

In [0]:
# Load Train

path = 'flower_data/valid/'
data = []
labels = []
data, labels = read_images(path)

In [10]:
print(type(data))
print(data.shape)
print(labels.shape)

<class 'numpy.ndarray'>
(818,)
(818, 102)


In [0]:
#Plotando uma das imagens
#plt.imshow(data[0].numpy().squeeze(), cmap = 'Greys_r')